In [1]:
# modules
# import basics
import os
# import re
import csv
import json
from termcolor import colored
# import time
# import requests
# import numpy as np
# import pandas as pd

# from collections import defaultdict
# import itertools as it

# from time import sleep
# from datetime import datetime, timedelta 
from dotenv import load_dotenv
# from matplotlib import pyplot as plt

# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import CountVectorizer
#from requests.auth import HTTPBasicAuth

# text processing dependencies
# from bs4 import BeautifulSoup
# from boilerpy3 import extractors

# mysql dependencies
import mysql.connector

In [2]:
# initialize variables

load_dotenv(override=True)

# mysql credentials
PASSWORD = os.getenv("PASSWORD")
USER = os.getenv("USER")

In [3]:
# connect to db
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user=USER,
  password=PASSWORD,
  database="mpp21"
)

mycursor = mydb.cursor()

In [4]:
# read newspaper info 

newspapers = {}

with open('./newspapers-collected.csv', 'r') as f:
        reader = csv.reader(f)
        for r in reader:
            newspapers[r[0]] = r[1]

In [5]:
# TEST IF ONLY ABSTRACT
# db methods
def select_test(curs, db, n):
    
    curs.execute("""SELECT article_id, content 
                FROM articles 
                WHERE newspaper = '""" + n + """' 
                AND year(Date) != '""" + str(2010) +"""'
                AND is_privacy != 'duplicate'
                """)
                #LIMIT 200""") 
    
    arts = {}
    result = curs.fetchall()
    for r in result:
        arts[r[0]] = {
            "content": r[1]
        }
    
    return arts

In [122]:
#for n in newspapers.keys():
abstract = select_test(mycursor, mydb, "NYT")
abstract_contents = {}
full_texts = {}
for i in abstract.keys():
    if abstract[i]["content"][:8] == "ABSTRACT":
        abstract_contents[i] = abstract[i]["content"]
    else:
        full_texts[i] = abstract[i]["content"]

In [6]:
# db methods
def select_summary(curs, db, n):
    
    curs.execute("SELECT article_id, title, summary, metadata, content FROM articles WHERE newspaper = '" + n + "' LIMIT 500") 
    
    arts = {}
    result = curs.fetchall()
    for r in result:
        arts[r[0]] = {
            "title": r[1],
            "summary": json.loads(r[2]), 
            "wordCount": json.loads(r[3])["wordCount"], 
            "content": r[4]
        }
    
    return arts

In [7]:
sample = select_summary(mycursor, mydb, "TG")

In [112]:
# def filter_sentence(t):
    
#     tokens = word_tokenize(t)
#     filtered_sentence = [w for w in tokens if not w in stop_words]  
    
#     return filtered_sentence

In [113]:
# from nltk.corpus import stopwords  
# from nltk.tokenize import word_tokenize  
  
# stop_words = set(stopwords.words('english'))  


In [8]:
import random

# db methods
def select_annotation_summary(curs, db, n):
    
    curs.execute("""SELECT article_id, title, summary, metadata, content 
                FROM articles 
                WHERE newspaper = '""" + n + """' 
                AND year(Date) != '""" + str(2010) +"""'
                AND is_privacy != 'duplicate'
                """) 
    
    arts = {}
    result = curs.fetchall()
    for r in result:
        arts[r[0]] = {
            "title": r[1],
            "summary": json.loads(r[2]), 
            "wordCount": json.loads(r[3])["wordCount"], 
            #"content": r[4]
        }
    
    return arts

def get_annotation_sample(prc, keys):
    
    total = prc * len(keys) / 100
    copy = keys
    
    count = 0
    random_keys = []
    
    while count < total:
        
        count += 1
        choice = random.choice(copy)
        random_keys.append(choice)
        copy.remove(choice)
        
    return total, sorted(random_keys)

In [9]:
articles = {}
annotations = {}

for n in newspapers.keys():

    all_articles = select_annotation_summary(mycursor, mydb, n)
    articles[n] = all_articles
    annotation_sample = get_annotation_sample(5, list(all_articles.keys()))
    annotations[n] = {
        "count": annotation_sample[0],
        "articles": annotation_sample[1]
    }

In [10]:
def print_article(i, a):
    
    print(colored(i, "red", attrs=['bold']))
    print(colored(a["title"], "blue", attrs=['bold']))
    print()
    
    print(colored("TOTAL WORDS: "+str(a["wordCount"])+"", "blue", attrs=['bold']))

    print(colored("TOTAL TERMS: "+str(len(a["summary"]["terms"]))+"", "blue", attrs=['bold']))
#     #print(colored("W-to-T ratio: "+str(len(sample[a]["summary"]["terms"]) / len(filter_sentence(sample[a]["content"])))+"", "blue", attrs=['bold']))

    
    print()
    print(colored("EXTRACT", "blue", attrs=['bold']))
    print(a["summary"]["extract"])
    print()
    print(colored("TERM(S)", "blue", attrs=['bold']))

    ct = 1
    for term in a["summary"]["terms"]:
        print(colored("("+str(ct)+")", "blue", attrs=['bold']), term)
        print()
        ct += 1
    print()
    print()
    print(colored("==" * 80, 
                  "red", attrs=['bold']))
    print()
    print()
    print()
    
    return

In [11]:
n = "TG"
for i in annotations[n]["articles"]:
    print_article(i, articles[n][i])
    


TG_100
Tribute comfort for Speed's family: Welsh and English fans pay respects to manager Family asks for privacy as inquest begins today

TOTAL WORDS: 498
TOTAL TERMS: 2

EXTRACT
... privacy as inquest begins today Helen Carter The family of Gary Speed has said that they are "overwhelmed" by the outpouring of tributes towards the Wales football manager, whose death will be the subject of an inquest set to open today. Nicholas Rheinberg, the coroner, confirmed through a spokesman that his office had been informed of the death by police. The inquest will be opened at the coroners court at Warrington town hall before a full hearing takes place at a later date. The body of  Speed ...

TERM(S)
(1) ... Welsh and English fans pay respects to manager Family asks for privacy as inquest begins today Helen Carter The family of Gary Speed ...

(2) ... ask that the family are now given the respect of some privacy to just grieve on their own." At a later date, the ...






TG_10101
What's the best

(6) ... work constructively with the City of Los Angeles while protecting the privacy of our riders." The city has strict privacy and confidentiality standards, and the data is not subject to public ...






TG_12260
Ben Stokes attacks 'despicable' Sun story about family tragedy;England cricketer says front-page article about family trauma in 1980s is immoral

TOTAL WORDS: 712
TOTAL TERMS: 3

EXTRACT
... privacy of his family members. He added: "This is the lowest form of journalism, focused only on chasing sales with absolutely no regard for the devastation caused to lives as a consequence. It is totally out of order." Tom Harrison, the England and Wales Cricket Board chief executive, also condemned the decision to publish the article. "We, like the wider sporting world, are disgusted and appalled at the actions taken in revealing the tragic events from Ben's past. We are saddened that an intrusion ...

TERM(S)
(1) ... only on chasing sales" and asked people to respect his family's p

TOTAL TERMS: 7

EXTRACT
... Privacy International's human rights and technology adviser Eric King said. "Skype's misleading security assurances continue to expose users around the world to unnecessary and dangerous risk. It's time for Skype to own up to the reality of its security and to take a leadership position in global communications." In response, Skype said: "Privacy International has not been in touch with us so it will take us some time to read and digest the report before we are in a position to respond. We will ...

TERM(S)
(1) ... then vulnerable users will continue to be exposed to avoidable risks," Privacy International's human rights and technology adviser Eric King said. "Skype's misleading ...

(2) ... a leadership position in global communications." In response, Skype said: " Privacy International has not been in touch with us so it will ...

(3) ... and aims to provide users with the best possible levels of privacy and security." Skype's system offers free peer-to-pe

TG_4093
US scrambles to find Edward Snowden and urges Russia to co-operate

TOTAL WORDS: 1473
TOTAL TERMS: 1

EXTRACT
... But in its strongly worded front-page commentary the People's Daily countered that Snowden's actions had "torn off Washington's sanctimonious mask". "In a sense the  United States  has gone from a 'model of human rights' to 'an eavesdropper on personal privacy', the 'manipulator' of the centralised power over the international Internet, and the mad 'invader' of other countries' networks," the official Communist party paper said. Carney said the  US  was working on the assumption that Snowden was  ...

TERM(S)
(1) ... from a 'model of human rights' to 'an eavesdropper on personal privacy ', the 'manipulator' of the centralised power over the international Internet, ...






TG_4106
Weekend: GARDENS: Small is beautiful: Got a tiny back garden that needs a revamp? This compact courtyard will give you big ideas, says Jane Perrone. Photograph by Sophia Evans

TOTAL WORD

TOTAL TERMS: 1

EXTRACT
...  student has admitted to hanging a noose from a tree on campus, a day after hundreds attended a rally there decrying racism at the university. University police and the office of student affairs said on Thursday that they had identified the student but would not release a name due to federal privacy laws. The student is "no longer on campus", a police spokesman, Keith Lawrence, said, adding that the person "will be subject to Duke's student conduct process". "Duke continues to coordinate with state  ...

TERM(S)
(1) ... the student but would not release a name due to federal privacy laws. The student is "no longer on campus", a police spokesman, ...






TG_6774
Eat. Nourish. Glow. by Amelia Freer - digested read;John Crace reduces the latest dietary pep talk by nutritionist to the stars Amelia Freer to a cleansing 600 words

TOTAL WORDS: 643
TOTAL TERMS: 1

EXTRACT
This isn't about me, though. This is about you. The journey each and every one of us - inclu

In [12]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>

In [13]:
!jupyter nbconvert 4-privacy-filter.ipynb --no-input --no-prompt --output "./htmls/tg_final.html"

[NbConvertApp] Converting notebook 4-privacy-filter.ipynb to html
[NbConvertApp] Writing 1355223 bytes to ./htmls/tg_final.html


In [281]:
all_ids = []

for n in newspapers.keys():
    for i in annotations[n]["articles"]:
        all_ids.append(i)

In [283]:
len(set(all_ids))

2663

In [284]:
with open('to_annotate.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerows(sorted(all_ids))

In [285]:
import pandas as pd

In [286]:
df = pd.DataFrame(sorted(all_ids))

In [287]:
df.to_csv("htmls/to_annotate.csv", index=False, header=False)

In [333]:
!zip -r annotations.zip htmls

updating: htmls/ (stored 0%)
updating: htmls/.ipynb_checkpoints/ (stored 0%)
updating: htmls/tg.html (deflated 79%)
updating: htmls/tdp.html (deflated 82%)
updating: htmls/nzh.html (deflated 81%)
updating: htmls/nyt.html (deflated 81%)
updating: htmls/usa.html (deflated 81%)
updating: htmls/afr.html (deflated 82%)
updating: htmls/smh.html (deflated 81%)
updating: htmls/to_annotate.csv (deflated 70%)
updating: htmls/ts.html (deflated 81%)
updating: htmls/dt.html (deflated 80%)
updating: htmls/gm.html (deflated 81%)


In [340]:
# for n in newspapers.keys():
#     print(n, "\t\t", len(annotations[n]["articles"]))

In [37]:
from random import randint

In [164]:
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&amp;*_~'''
 
def rem_punct(a):
    
 
    no_punc = ""
    for char in a:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc
 

In [268]:
sample_dump = {}

for i in sample.keys():
    sample_dump[i] = {
        "text": rem_punct(sample[i]["title"]),# + sample[i]["content"],
        "label": randint(0,1),
        "alpha": "a"
    }

In [269]:
# with open("sample_NYT.json", "w") as outfile:  
#     json.dump(sample_dump, outfile) 

In [270]:
df = pd.DataFrame.from_dict(sample_dump, orient="columns").T#=["id", "alpha", "label", "text"]).T#, orient="index")

In [211]:
## BERT

In [210]:
# import torch# If there's a GPU available...
# if torch.cuda.is_available():        # Tell PyTorch to use the GPU.    
#     device = torch.device("cuda")    
#     print('There are %d GPU(s) available.' % torch.cuda.device_count())    
#     print('We will use the GPU:', torch.cuda.get_device_name(0))# If not...
# else:
#     print('No GPU available, using the CPU instead.')
#     device = torch.device("cpu")

In [199]:
# import pandas as pd

# #df = pd.read_csv("privacy.csv")
# print('Number of training sentences: {:,}\n'.format(df.shape[0]))
# df.sample(10)

In [273]:
# Get the lists of sentences and their labels.
sentences = df.text.values
labels = df.label.values

In [200]:
# from transformers import BertTokenizer# Load the BERT tokenizer.
# print('Loading BERT tokenizer...')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [201]:
# # Tokenize all of the sentences and map the tokens to thier word IDs.
# input_ids = []# For every sentence...
# for sent in sentences:
#     # `encode` will:
#     #   (1) Tokenize the sentence.
#     #   (2) Prepend the `[CLS]` token to the start.
#     #   (3) Append the `[SEP]` token to the end.
#     #   (4) Map tokens to their IDs.
#     encoded_sent = tokenizer.encode(
#                         sent,                      # Sentence to encode.
#                         add_special_tokens = True, # Add '[CLS]' and '[SEP]'                        # This function also supports truncation and conversion
#                         # to pytorch tensors, but we need to do padding, so we
#                         # can't use these features :( .
#                         #max_length = 512,          # Truncate all sentences.
#                         #return_tensors = 'pt',     # Return pytorch tensors.
#                    )
    
#     # Add the encoded sentence to the list.
#     input_ids.append(encoded_sent)# Print sentence 0, now as a list of IDs.
# print('Original: ', sentences[0])
# print('Token IDs:', input_ids[0])

In [202]:
# print('Max sentence length: ', max([len(sen) for sen in input_ids]))

In [203]:
# # We'll borrow the `pad_sequences` utility function to do this.
# from keras.preprocessing.sequence import pad_sequences# Set the maximum sequence length.
# # I've chosen 64 somewhat arbitrarily. It's slightly larger than the
# # maximum training sentence length of 47...
# MAX_LEN = 32###64
# print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
# print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))# Pad our input tokens with value 0.
# # "post" indicates that we want to pad and truncate at the end of the sequence,
# # as opposed to the beginning.
# input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
#                           value=0, truncating="post", padding="post")
# print('\Done.')

In [278]:
# Create attention masks
attention_masks = []# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [279]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split# Use 90% for training and 10% for validation.

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [280]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(np.array(list(train_labels))) #train_labels)
validation_labels = torch.tensor(np.array(list(validation_labels)))

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [281]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.
batch_size = 32# Create the DataLoader for our training set.

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [282]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 4,#2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
#model.cuda()

In [204]:
# # Get all of the model's parameters as a list of tuples.
# params = list(model.named_parameters())
# print('The BERT model has {:} different named parameters.\n'.format(len(params)))
# print('==== Embedding Layer ====\n')
# for p in params[0:5]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
# print('\n==== First Transformer ====\n')

# for p in params[5:21]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
# print('\n==== Output Layer ====\n')

# for p in params[-4:]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [284]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup# Number of training epochs (authors recommend between 2 and 4)
epochs = 4# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [285]:
import numpy as np# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [286]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [205]:
# import random# This training code is based on the `run_glue.py` script here:
# # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# # Set the seed value all over the place to make this reproducible.
# seed_val = 42
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)# Store the average loss after each epoch so we can plot them.
# loss_values = []# For each epoch...

# for epoch_i in range(0, epochs):
    
#     # ========================================
#     #               Training
#     # ========================================
    
#     # Perform one full pass over the training set.    print("")
#     print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
#     print('Training...')    # Measure how long the training epoch takes.
#     t0 = time.time()    # Reset the total loss for this epoch.
#     total_loss = 0    # Put the model into training mode. Don't be mislead--the call to 
#     # `train` just changes the *mode*, it doesn't *perform* the training.
#     # `dropout` and `batchnorm` layers behave differently during training
#     # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
#     model.train()    # For each batch of training data...
#     for step, batch in enumerate(train_dataloader):        # Progress update every 40 batches.
#         if step % 40 == 0 and not step == 0:
#             # Calculate elapsed time in minutes.
#             elapsed = format_time(time.time() - t0)
            
#             # Report progress.
#             print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))        # Unpack this training batch from our dataloader. 
#         #
#         # As we unpack the batch, we'll also copy each tensor to the GPU using the 
#         # `to` method.
#         #
#         # `batch` contains three pytorch tensors:
#         #   [0]: input ids 
#         #   [1]: attention masks
#         #   [2]: labels 
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)        # Always clear any previously calculated gradients before performing a
#     # backward pass. PyTorch doesn't do this automatically because 
#         # accumulating the gradients is "convenient while training RNNs". 
#         # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
#         model.zero_grad()                # Perform a forward pass (evaluate the model on this training batch).
#         # This will return the loss (rather than the model output) because we
#         # have provided the `labels`.
#         # The documentation for this `model` function is here: 
#         # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
#         outputs = model(b_input_ids, 
#                     token_type_ids=None, 
#                     attention_mask=b_input_mask, 
#                     labels=b_labels)
        
#         # The call to `model` always returns a tuple, so we need to pull the 
#         # loss value out of the tuple.
#         loss = outputs[0]        # Accumulate the training loss over all of the batches so that we can
#         # calculate the average loss at the end. `loss` is a Tensor containing a
#         # single value; the `.item()` function just returns the Python value 
#         # from the tensor.
#         total_loss += loss.item()        # Perform a backward pass to calculate the gradients.
#         loss.backward()        # Clip the norm of the gradients to 1.0.
#         # This is to help prevent the "exploding gradients" problem.
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)        # Update parameters and take a step using the computed gradient.
#         # The optimizer dictates the "update rule"--how the parameters are
#         # modified based on their gradients, the learning rate, etc.
#         optimizer.step()        # Update the learning rate.
#         scheduler.step()    # Calculate the average loss over the training data.
#     avg_train_loss = total_loss / len(train_dataloader)            
    
#     # Store the loss value for plotting the learning curve.
#     loss_values.append(avg_train_loss)    
#     print("")
#     print("  Average training loss: {0:.2f}".format(avg_train_loss))
#     print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
#     # ========================================
#     #               Validation
#     # ========================================
#     # After the completion of each training epoch, measure our performance on
#     # our validation set.    print("")
#     print("Running Validation...")    
#     t0 = time.time()    # Put the model in evaluation mode--the dropout layers behave differently
#     # during evaluation.
#     model.eval()    # Tracking variables 
#     eval_loss, eval_accuracy = 0, 0
#     nb_eval_steps, nb_eval_examples = 0, 0    # Evaluate data for one epoch
#     for batch in validation_dataloader:
        
#         # Add batch to GPU
#         batch = tuple(t.to(device) for t in batch)
        
#         # Unpack the inputs from our dataloader
#         b_input_ids, b_input_mask, b_labels = batch
        
#         # Telling the model not to compute or store gradients, saving memory and
#         # speeding up validation
#         with torch.no_grad():                    # Forward pass, calculate logit predictions.
#             # This will return the logits rather than the loss because we have
#             # not provided labels.
#             # token_type_ids is the same as the "segment ids", which 
#             # differentiates sentence 1 and 2 in 2-sentence tasks.
#             # The documentation for this `model` function is here: 
#             # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
#             outputs = model(b_input_ids, 
#                             token_type_ids=None, 
#                             attention_mask=b_input_mask)
        
#         # Get the "logits" output by the model. The "logits" are the output
#         # values prior to applying an activation function like the softmax.
#         logits = outputs[0]        # Move logits and labels to CPU
#         logits = logits.detach().cpu().numpy()
#         label_ids = b_labels.to('cpu').numpy()
        
#         # Calculate the accuracy for this batch of test sentences.
#         tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
#         # Accumulate the total accuracy.
#         eval_accuracy += tmp_eval_accuracy        # Track the number of batches
#         nb_eval_steps += 1    # Report the final accuracy for this validation run.
#     print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
#     print("  Validation took: {:}".format(format_time(time.time() - t0)))
#     print("")
# print("Training complete!")

In [206]:
# # import plotly.express as px
# f = pd.DataFrame(loss_values)
# f.columns=['Loss']
# f
# # fig = px.line(f, x=f.index, y=f.Loss)
# # fig.update_layout(title='Training loss of the Model',
# #                    xaxis_title='Epoch',
# #                    yaxis_title='Loss')
# # fig.show()